In [1]:
import os
import requests
import operator
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.callbacks import get_openai_callback

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_PROJECT"] = "RRL project"

os.environ["AZURE_OPENAI_API_KEY"] = "EhMIoJnOsNomEJ8TRfOEoIc1jC49AwdEwmZ8UDi4lh6dsUZ4WEArJQQJ99BAACYeBjFXJ3w3AAAAACOGl3Rt"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://week31004687013.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview"


### Hotel info

In [2]:
HOTEL_INFO = """Hilton Orrington / Evanston
3.5 starsBased on 380 reviews

1710 Orrington AveEvanstonIllinoisUnited States60201 See Map

ReservationDesk.com:866-215-2318
Hotel Description
Take advantage of recreation opportunities such as a 24-hour fitness center, or other amenities including complimentary wireless internet access and concierge services. Additional amenities at this hotel include wedding services, a fireplace in the lobby, and discounted use of a nearby fitness facility.

Enjoy a satisfying meal at Evanston Corner Bistro serving guests of Hilton Orrington / Evanston. Wrap up your day with a drink at the bar/lounge. Cooked-to-order breakfasts are served on weekdays from 7:30 AM to 10:00 AM and on weekends from 8:00 AM to 11:00 AM for a fee.

The property will be renovating from July 1 2024 to September 6 2024 (completion date subject to change). The following areas are affected: Lobby During renovations, the hotel will make every effort to minimize noise and disturbance.

Featured amenities include a business center, express check-in, and express check-out. Planning an event in Evanston? This hotel has 15756 square feet (1464 square meters) of space consisting of a conference center and meeting rooms.

Make yourself at home in one of the 269 guestrooms featuring refrigerators and flat-screen televisions. Your pillowtop bed comes with down comforters and premium bedding. Complimentary wireless internet access keeps you connected, and cable programming is available for your entertainment. Private bathrooms have designer toiletries and hair dryers.

You'll be centrally located in Evanston with a stay at Hilton Orrington / Evanston, within a 15-minute walk of Halim Time & Glass Museum and Clark Street Beach. This hotel is 0.7 mi (1.2 km) from Mary and Leigh Block Museum of Art and 1 mi (1.6 km) from Northwestern University.

Near Northwestern University

Bosnian, Arabic, Russian, English, Croatian, Romanian, Spanish, Serbian

Visa, Diners Club, Debit cards, Discover, Cash, American Express, JCB International, Mastercard

Hotel Policies

Know Before You Go

Only registered guests are allowed in the guestrooms. Parking height restrictions apply. This property welcomes guests of all sexual orientations and gender identities (LGBTQ+ friendly).


Age Restriction

Guests under the age of 21 can only check in with a parent or official guardian.



Check-In Instructions

Extra-person charges may apply and vary depending on property policy
Government-issued photo identification and a credit card, debit card, or cash deposit may be required at check-in for incidental charges
Special requests are subject to availability upon check-in and may incur additional charges; special requests cannot be guaranteed
This property accepts credit cards, debit cards, and cash
Long-term renters welcome
Safety features at this property include a fire extinguisher, a security system, and a first aid kit
Please note that cultural norms and guest policies may differ by country and by property; the policies listed are provided by the property

Check-In Special Instructions

Front desk staff will greet guests on arrival.


Fees

Optional

Fee for cooked-to-order breakfast: approximately USD 20 for adults and USD 20 for children
Covered valet parking fee: USD 33.00 per night (in/out privileges)
Pet fee: USD 75 per accommodation, per stay
Service animals are exempt from fees
In-room microwave fee: USD 25 per night
Rollaway bed fee: USD 25.0 per night
The above list may not be comprehensive. Fees and deposits may not include tax and are subject to change.

- Read less
Amenities
Internet Access
Spa
Fitness Center
Restaurant
Pets Allowed (conditions apply)
Room Service
Accessible
Meeting Facilities
Bar
Smoke free
Smoking Areas
24 hour
Concierge
Conference Center



Based on 380 User Reviews
Search reviews

Newest to Oldest

Traveler Type
4 stars review
Young Couple
Dec 2024
Ben
lobby re-do is great
rooms need a facelift - old and tired especially for parents weekend price!
5 stars review
Family with Younger Children
Dec 2024
Crystal
it was an easy walk to the university and downtown evanston. the room was huge, well equipped, and very comfortable.
nothing. it was great!
1 star review
Business Traveler
Nov 2024
Ian
check out.
considering i booked several months in advance i was only informed about refurbishment work between june and 31st december a couple of days before arrival. noise from builders until 8 pm, breakfast not available before 07:30, wasn't informed before hand, never got to eat a breakfast i had booked.
0 star review
Family with Younger Children
Oct 2024
Julie
the location was great, but not for the price.
before we arrived, the hotel asked to change our room from 2 beds to 1 bed because, after i booked, they booked several sports teams that needed rooms with 2 beds. i explained that we needed 2 beds because we were traveling with children. when we arrived, they put us in a room directly under loud construction noise and refused to move us to a different floor. we are hilton honors members and they also refused to upgrade us or discount in any way to address the inhabitable room. the only thing they offered to make us more comfortable was free water.
3 stars review
Family with Younger Children
Oct 2024
Ruth
great location right next to our daughter's university so we could walk to her dorm and move her in. right next to all of the stores and restaurants.
a little expensive.
4 stars review
Solo Traveler
Sep 2024
Colleen
great location, clean and comfortable.
3 stars review
Business Traveler
Sep 2024
Aline
the room is very comfortable
the food service one day didn’t bring one thing that i ordered, and i had no way to complain. the hotel was under construction, and both mornings they started constructing a 7am and 7:30 am.
3 stars review
Family with Younger Children
Sep 2024
Vicky
the lobby is under renovation, making entry and exit inconvenient.
0 star review
Family with Younger Children
Sep 2024
Christopher
staff was friendly and helpful.
upon arrival to the room it looked like it had been chewed up by a pack of dogs. seeing as we had a dog of our own that had opted for a pet stay we decided to overlook that, however immediately we noticed the air conditioner was not cooling the room. we turned it down to 62 and waited a while.. meanwhile i noticed the giant sheets of particle board under the mattresses instead of box springs. also worth noting that though they had noted they were going under lobby renovations we kept having fire drills. ultimately the room would not cool down and we asked to be moved to a different room and we're told that there was no rooms available and they would send up a technician. technician came around 10pm and confirmed the ac was blowing at 61 and that the room would cool down within an hour. at 7:00 a.m. the next morning the room had not cooled down 1°, still at 72. we couldn't sleep at all as we both run hot and prefer temperatures in the 66 to 68 range for sleeping. we only stayed one night of our two nights and i opted for somewhere that wasn't a complete and total waste of time and money.
4 stars review
Solo Traveler
Aug 2024
Brandon
the room
paying another deposit after already prepaying the room!
3 stars review
Business Traveler
Aug 2024
Charles
location
contraction during my stay (loud noise) wifi failure. i had to go to starbucks to work while i would have worked at the property.
1 star review
Business Traveler
Jun 2024
Pan
not really
not desk lamp, call the front desk to request add didn’t get it after 2 days. food there is also not good
1 star review
Family with Younger Children
Jun 2024
Jennifer
the room wasn't the cleanest and the chair in the room had rips all over it.
4 stars review
Solo Traveler
Jun 2024
Melinda
easy, responsive staff. terribly kind and accommodating and professional.
nope; everything was perfect.
3 stars review
Family with Younger Children
May 2024
Smith
location was perfect, downtown evanston. there was a local restaurant near by
there were only 2 outlets in the room
3 stars review
Family with Younger Children
May 2024
Kathy
the breakfast was very good. the staff was friendly and accommodating. beds were comfortable.
this hotel is apparently remodeling several floors. we were assigned to an older floor and arrived to a hot/stuffy room. upon returning from sightseeing and dinner, the a/c still didn't work though maintenance had reported it as fixed. the front desk was able to provide us a fan that made sleeping that night more bearable. after 2nd day full of activities in chicago, arrived back to hotel around midnight to discover a/c still not working. after more late night maintenance visits, staff determined a/c could not be fixed that night. thankfully around 1 am we were moved to an upgraded room at no additional charge with working a/c!! the night manager also removed room charges for that 1st night's stay which we appreciated. the front desk, maintenance, breakfast staffs, etc. were all very nice and courteous. the morning of our last vacation day, we again headed to chicago before housekeeping serviced our room. carelessly, i left my 2nd phone laying on the desk. when we returned to the room that evening, the room had been serviced, but my phone was no longer on the desk. unfortunately i didn't realize the phone was really missing until we checked out the next morning, returned home and was unable to find it in any of our bags, etc. i called the front desk and was told housekeeping did not report any found items left behind. looking back, none of us saw the phone sitting on the desk after returning from chicago (that third sunday night of our stay). i told the front desk i felt confident that it disappeared during our stay and that i would be leaving a review to reflect that. i guess i should have tried leaving it in the safe....but the safe in our 1st room didn't work anyway so i guess we didn't think about trying to use the safe in our 2nd room. i am so disappointed as i had been very happy with how friendly staff had been. i really miss my phone and would like to know where it went.
3 stars review
Business Traveler
May 2024
Anonymous
first 2 nights ac unit of the building not the room, noise drive me crazy at night. changed to the other side for extra 2 night better but the sound proof is not good enough neither.
1 star review
Business Traveler
May 2024
Anonymous
it was close to the college
you had to pay for valet parking and for being a nice hotel the carpet was coming apart when you go in to the bathroom.
5 stars review
Business Traveler
May 2024
Anonymous
beautiful hotel in lovely area.
would love to see free coffee in libby starting at 6am for those on the go - not a fan of the taste of the in-room coffee maker etc
4 stars review
Apr 2024
Mike
staff were extremely friendly and accommodating. property is in walking distance of campus and restaurants.
room was small, especially the bathroom.
4 stars review
Apr 2024
Verified Hotel Guest
the room is clean and comfortable.
the staff at the registration counter is not that friendly.
SEE MAP
Map
COMPARE HOTELS NEARBY
Holiday Inn Chicago North-Evanston, an IHG Hotel
  0.3 
3.75 stars
 Based on 784 reviews
Hilton Garden Inn Chicago North Shore/Evanston
  0.3 
4.05 stars
 Based on 481 reviews
Hyatt House Chicago/Evanston
  0.2 
4.25 stars
 Based on 349 reviews
Graduate by Hilton Evanston
  0.3 
4.1 stars
 Based on 600 reviews
The Gem Museum Suites
  0.4 
 

"""

In [3]:
HOTEL_INFO = """
Stay next to Northwestern University
Our hotel is located in the Chicago North Shore District and 17 miles to OHare International Airport. We’re adjacent to Northwestern University. Lake Michigan beaches and parks and the commuter rail link to downtown Chicago are a short walk away. We offer expansive meeting space, a 24-hour business center, free WiFi, and a free area shuttle.
Our amenities
Connecting Rooms
Free WiFi
Non-smoking rooms
Digital Key
EV charging
On-site restaurant
Indoor pool
Fitness center
Pet-friendly rooms
Business center
Meeting rooms
Hilton Honors member benefits
Hilton Honors Discount rate
Points toward free nights and more
Choose Your Room
Digital Check-In
Hilton Honors Experiences
Digital Key
Hotel policies
Parking
Pets
Smoking
WiFi
Self-parking
$24.00 per day
Valet parking
Not available
EV charging
On-site
Secured
Available
Covered
Available
In/Out privileges
Available
Other parking information
Secured Covered Parking Garage adjacent to hotel.
Rooms and suites
Your stay includes
✓ Free WiFi
✓ Non-smoking rooms
✓ On-site restaurant
✓ Indoor pool
✓ Fitness center
✓ Pet-friendly rooms
✓ Business center
All
Guest Rooms
Suites
Accessible
This hotel offers Confirmed Connecting Rooms, subject to availability.
9 rooms available for your selected dates. We're showing the average price per night.
Select currency
Currency disclaimer
AED - UAE DirhamAFN - Afghanistan AfghaniALL - Albania LekeAMD - Armenian DramANG - Dutch GuilderAOA - Angolan KwanzaARS - Argentine PesoAUD - Australian DollarsAWG - Aruban FlorinAZN - Azerbaijan New ManatBAM - Bosnian MarkBBD - Barbadian DollarBDT - Bangladeshi TakaBGN - Bulgarian LevBHD - Bahraini DinarBIF - Burundi FrancBMD - Bermudian DollerBND - Brunei DollarBOB - Bolivian BolivianoBRL - Brazilian RealsBSD - Bahamian DollarBTN - Bhutan NgultrumBWP - Botswana PulaBYN - New Belarusian RubleBZD - Belize DollarCAD - Canadian DollarCDF - Congolese FrancCHF - Swiss FrancCLF - Unidad de FomentoCLP - Chilean PesoCNY - Chinese Yuan RenminbiCOP - Colombian PesoCRC - Costa Rican ColonCUC - Peso ConvertibleCUP - Cuban PesoCVE - Cape Verde EscudoCZK - Czech KorunaDJF - Djibouti FrancDKK - Danish KroneDOP - Dominican PesosDZD - Algerian DinarEGP - Egyptian PoundERN - Eritrea NakfaETB - Ethiopian BirrEUR - EuroFJD - Fiji DollarFKP - Falkland Islands PoundGBP - British PoundGEL - Georgian LariGHS - Ghana CediGIP - Gibraltar PoundGMD - Gambian DalasiGNF - Guinea FrancGTQ - Guatemalan QuetzalGYD - Guyanese DollarHKD - Hong Kong DollarHNL - Honduran LempiraHRK - Croatian KunaHTG - Haitian GourdeHUF - Hungarian ForintIDR - Indonesian RupiahILS - Israeli New ShekelINR - Indian RupeeIQD - Iraqi DinarIRR - Iranian RialISK - Iceland KronaJMD - Jamaican DollarJOD - Jordanian DinarJPY - Japanese YenKES - Kenyan ShillingKGS - Kyrgyzstan SomKHR - Cambodian RielKMF - Comoros FrancKPW - North Korean WonKRW - South Korean WonKWD - Kuwaiti DinarKYD - Cayman Islands DollarKZT - Kazakhstan TengeLAK - Lao KipLBP - Lebanese PoundsLKR - Sri Lanka RupeeLRD - Liberian DollarLSL - Lesotho LotiLYD - Libyan DinarMAD - Moroccan DirhamMDL - Moldovan LeuMGA - Malagasy AriaryMKD - Macedonian DenarMMK - Myanmar KyatMNT - Mongolian TugrikMOP - Macau PatacaMRO - Mauritanian OuguiyaMUR - Mauritius RupeeMVR - Maldive RufiyaaMWK - Malawi KwachaMXN - Mexican PesoMYR - Malaysian RinggitMZN - Mozambique MeticalNAD - Namibia DollarNGN - Nigerian NairaNIO - Nicaraguan Cordoba OroNOK - Norwegian KroneNPR - Nepalese RupeeNZD - New Zealand DollarOMR - Omani RialsPAB - Panamanian BalboaPEN - Peruvian Nuevo SolPGK - Papua New Guinea KinaPHP - Philippine PesoPKR - Pakistan RupeePLN - Polish ZlotyPYG - Paraguay GuaraniQAR - Qatari RialRON - Romanian New LeiRSD - Serbian DinarRUB - Russian RubleRWF - Rwandan FrancSAR - Saudi RiyalSBD - Solomon Islands DollarSCR - Seychelles RupeeSDG - Sudanese PoundSEK - Swedish KronaSGD - Singapore DollarSHP - St. Helena PoundSLL - Sierra Leone LeoneSOS - Somali ShillingSRD - Suriname DollarSTD - Sao Tome/Principe DobraSVC - El Salvador ColonSYP - Syrian PoundSZL - LilangeniTHB - Thai BahtTJS - Tajikistan SomoniTMT - Turkmenistan New ManatTND - Tunisian DinarTOP - Tonga Pa''angaTRY - Turkish LiraTTD - Trinidad/Tobago DollarTWD - New Taiwan DollarTZS - Tanzanian ShillingUAH - Ukraine HryvniaUGX - Uganda ShillingUSD - US DollarUYU - Uruguayan PesoUZS - Uzbekistan SumsVEF - Venezuelan Bolivar FuerteVND - Vietnamese DongVUV - Vanuatu VatuWST - Samoan TalaXAF - CFA Franc BeacXCD - East Caribbean DollarXOF - CFA Franc BCEAOXPF - CFP FrancYER - Yemeni RialZAR - South African RandZMW - Zambian KwachaZWL - Zimbabwe Dollar


1 King Bed
1 King Bed
Room details >
Explore Rates From $182
Explore Rates From $182 for 1 King Bed


2 Double Beds Non Smoking
2 Double Beds Non Smoking
Room details >
Explore Rates From $182
Explore Rates From $182 for 2 Double Beds Non Smoking


1 King Mobility/hearing Accessible W/ Bathtub
1 King Mobility/hearing Accessible W/ Bathtub
Room details >
Explore Rates From $182
Explore Rates From $182 for 1 King Mobility/hearing Accessible W/ Bathtub

1 King Mobility/hearing Accessible Ri Shwr
1 King Mobility/hearing Accessible Ri Shwr
Room details >
Explore Rates From $182
Explore Rates From $182 for 1 King Mobility/hearing Accessible Ri Shwr


2 Double Mobility/hearing Access W/ Bathtub
2 Double Mobility/hearing Access W/ Bathtub
Room details >
Explore Rates From $182
Explore Rates From $182 for 2 Double Mobility/hearing Access W/ Bathtub


2 Double Hearing Accessible
2 Double Hearing Accessible
Room details >
Explore Rates From $182
Explore Rates From $182 for 2 Double Hearing Accessible


1 King Bed Corner Nonsmoking
1 King Bed Corner Nonsmoking
Room details >
Explore Rates From $201
Explore Rates From $201 for 1 King Bed Corner Nonsmoking


1 King Bed with Sofabed Nonsmoking
1 King Bed with Sofabed Nonsmoking
Room details >
Explore Rates From $201
Explore Rates From $201 for 1 King Bed with Sofabed Nonsmoking


1 King Bed Junior Suite with Sofabed Nosmoke
1 King Bed Junior Suite with Sofabed Nosmoke
Room details >
Explore Rates From $271
Explore Rates From $271 for 1 King Bed Junior Suite with Sofabed Nosmoke


1King Bed 1bdrm President Ste Sofabed Nonsmok
1King Bed 1bdrm President Ste Sofabed Nonsmok
Room details >
Sold out
This room is not available on your selected dates
Locations/
USA/
Illinois/
Evanston Hotels/
Hilton Garden Inn Chicago North Shore/Evanston/
Rooms
All options

Hilton Garden Inn Chicago North Shore/Evanston Official Site
$193
Visit site

Traveluro
$141
Visit site

Splitty
$141
Visit site

Super.com
$149
Visit site

Hotels In America
$156
Visit site

Brek
$157
Visit site

trivago DEALS
$168
Visit site

Hotels Ugogo
$182
Visit site

Expedia.com
$164
Visit site

Vio.com
$160
Visit site

PrimaStay
$182
Visit site

Priceline
$193
Visit site

Hktuyitrip
Nightly price with taxes + fees
$204
Visit site

Booketta
$182
Visit site

Booking.com
$193
Visit site

Tripadvisor.com
$164
Visit site

Hotels.com
$164
Visit site

Trip.com
$193
Visit site

goseek.com
$165
Visit site

BringFido
$164
Visit site

Travelocity.com
$164
Visit site

CheapTickets.com
$164
Visit site

Closest Hotel
$164
Visit site

Orbitz.com
$164
Visit site

pricetravel.com
$193
Visit site

Agoda
$193
Visit site

Despegar
$164
Visit site

eDreams
$164
Visit site

Bluepillow.com
$185
Visit site

Wego
$196
Visit site

LateRooms.com
$196
Visit site

Hotwire.com
$164
Visit site

trivago.com
$157
Visit site


Google review summary
4.2
793 reviews
5 star
4 star
3 star
2 star
1 star
Reviews on other travel sites

Tripadvisor
4/5 · 939 reviews

Priceline
8.1/10 · 96 reviews

Trip.com
3.9/5 · 135 reviews
Reviews
Write a review
All
Fitness (35)
Pool (14)
Parking (55)
Sleep (48)
Hot Tub (5)
Bathroom (31)
Gym (18)
Bar (29)
Family (15)
+ 18 more
Most helpful
All reviews
Angela H
2 months ago on  Tripadvisor
5/5
My family stayed for two nights and everything was just what we hoped for. All the staff we interacted with—front desk, restaurant, and housekeeping—were all very welcoming. …
Read more on Tripadvisor
Deal H
2 months ago on  Tripadvisor
2/5
Picked the hotel due to proximity to NWU and positive things about city of Evanston (vs Chicago downtown) along with the great reviews here.  …
Read more on Tripadvisor
SeoJoo Lee
a month ago on  Google
5/5
Business ❘ Family
I had a really nice during my visit to Chicago. It was winter, and the hot water worked perfectly, and the room was well-heated. The staffs and the housekeeping ladies were all very friendly too.
Hotel highlightsGreat value
Response from the owner
a month ago
Hello SeoJoo Lee, we're delighted to hear that you had a pleasant stay during your winter visit to Chicago. It's great to know that the hot water and room heating met your expectations and that our staff made a positive impression with their service. Thank you for sharing your experience with us. We look forward to welcoming you back in the future!
USA “Loyalty” Princess
3 weeks ago on  Google
3/5
Business
This place has racist assumptions. They treat customers based on race. It is a big crime when you live legally in the US and you can't get respect because you are a citizen but your problem is that you are not like the other race that runs … Read more
Patricia D
6 months ago on  Tripadvisor
5/5
We can't say enough good things about our hotel experience. The staff members are great! Very helpful with check-in, information about the area, arranging a taxi for us one night, and answering our questions. …
Read more on Tripadvisor
Rose Richardson
4 months ago on  Google
4/5
Vacation ❘ Family
Location was the best part! Very close to train station and nearby attractions. Enjoyed having the farmers market available on Saturday right outside the hotel! Rooms were dark and oddly set up for the suite.
Rooms3.0Service5.0Location5.0
Response from the owner
4 months ago
Thank you so much for staying with us, Rose! It was a pleasure having you and we are happy to see your 4 star review. We hope you stay with us at Hilton Garden Inn Chicago North Shore/Evanston next time you're visiting Evanston.
wendy halsey
2 months ago on  Google
5/5
Vacation ❘ Friends
Great location lots of walkable venues nearby for food, coffee and more. Raul at Front Desk was super professional, kind and helpful. Parking is next door in garage. Breakfast buffet was good as well all the right fresh options.
Rooms5.0Service5.0Location5.0
Hotel highlightsQuiet · Great value …Read more
Response from the owner
2 months ago
Hello Wendy, thank you for sharing your positive experience with us! We're delighted to hear that you enjoyed our convenient location and that Raul at the front desk made a great impression with his professionalism and kindness. It's also wonderful to know that you appreciated our breakfast buffet. We hope to welcome you back to Hilton Garden Inn soon for another enjoyable stay.
Grace Edwards
6 months ago on  Google
1/5
Vacation ❘ Family
The hotel itself is well maintained and clean, unfortunately their customer service left a lot lacking. I arrived at 6:45 to check in to our hotel room and they let me know they were still cleaning the room and it would be 10 minutes. After … Read more
Response from the owner
6 months ago
Hi Grace, thank you for your positive note about our hotel. However, we're truly sorry to hear about the issues you encountered during your stay and the inconvenience it caused, especially with a tired child. This is not the level of service we strive to provide at Hilton Garden Inn. Please know that we are looking into this matter to ensure it doesn't happen again. We encourage you to contact https://help.hilton.com/s/contactsupport and (847) 475-6400 to provide more details about your booking and further discuss your experience. We appreciate your feedback and hope to have the opportunity to better serve you in the future.
Christopher Huguelet
3 months ago on  Google
5/5
I was helped by a wonderful front desk worker, whose name was Al. He was kind, courteous, funny, and informative about the myriad of different activities at this hotel. I reccomend a stay at this location, if not for the kind service, than … Read more
Response from the owner
3 months ago
Hi Christopher, thank you for your wonderful review! We're thrilled to hear that Al made such a positive impression on you with his kindness and helpfulness. It's great to know you enjoyed our fitness center and jacuzzi as well. We look forward to welcoming you back to Hilton Garden Inn during your next visit to your relatives.
Lisa Langsdorf
3 months ago on  Google
1/5
Overcharged me for the pet and when I called to ask for a refund they said they’d have to talk to the manager to find out what’s going on. What’s going on is that you billed me 4x for the pet (4 nights) when I stayed 3 and was only supposed … Read more
Special Offers
Showing 1 - 10 of 10 offers
Showing 1 through 10 of 10 offers
Offer
Filters
0 filters selected

2X Points
Earn double Hilton Honors Points for each night of your stay.
Earn PointsHonors Member
View Details
, Opens new tab

Honors Discount Advance Purchase
Save up to 17% off our Best Available Rate* by booking in advance. It pays to plan ahead!
Discount
View Offer Details
, Opens new tab

Breakfast Included
Fuel your day with breakfast included for every registered guest, each night of your stay.
Kids and family
View Offer Details
, Opens new tab

AAA/CAA Members Save
AAA/CAA Members, Save up to 10%
Discount
View Details
, Opens new tab

Experience the Stay
Book our Experience the Stay package and indulge with an on-property credit and additional perks, like premium WiFi and early check-in.
Hotel credit
View Details
, Opens new tab

Military Family Rate
Active and retired military, veterans, and their families enjoy exclusive savings at participating hotels.
Discount
View Offer Details
, Opens new tab

Park & Stay
Get daily parking each night of your stay with our Park & Stay offer.
Parking
View Offer Details
, Opens new tab

Senior Rate
Seniors and their families can save up to 6% off our Best Available Rate* when traveling together.
Discount
View Details
, Opens new tab

Your journey starts with Points.
See how Hilton Honors can power your next stay. Go anywhere with Hilton Honors Points. No Blackout dates.
Earn PointsHonors Member
Reveal Offer
, Opens new tab

2X the Points, 2X the possibilities
Earn 2X Bonus Points on all stays through April 30 when you register for the Double Your Stay Promotion.
Earn PointsHonors Member
Register for Free
Page 1 of 1
Previous Page, 1 of 1






Response from the owner
3 months ago
Hello Lisa - We’re normally known for our exceptional attention to detail, but we're sorry about the issues you had regarding your charges and our service. We would like to have the opportunity to investigate this further and ask you to reach us at https://help.hilton.com/s/contactsupport or (847) 475-6400 so we can connect with you. Thanks for taking the time to share your feedback, and we look forward to hearing from you.


About this hotel
An 8-minute walk to Northwestern University, this contemporary hotel is a 0.6 miles from the beach on Lake Michigan.
All rooms and suites offer upscale decor and custom-designed beds, plus 32-inch flat-screen TVs with premium cable channels, microwaves, minifridges and free WiFi. Suites add separate living areas and pull-out sofas.
Continental and cooked-to-order breakfasts are available (fee), and there's an on-site restaurant serving American cuisine, as well as an unassuming bar and a 24/7 convenience store. There's also a fitness center and the indoor pool. Additional amenities include 3 meeting rooms and a business center.
Check-in time: 3:00 PM
Check-out time: 11:00 AM
Address & contact information
1818 Maple Ave, Evanston, IL 60201
(847) 475-6400
Website
Directions
Health & safety
Enhanced cleaning
Physical distancing
Minimized contact
Increased food safety
Health & safety measures are provided by the hotel. Visit their website for more details.
Amenities
Popular amenities
Pool
Wi-Fifree
Air conditioning
Breakfastextra charge
Internet
Wi-Fifree
Wi-Fi in public areas
Public internet workstation
Policies & payments
Smoke-free property
Credit cards
Debit cards
Cash
Services
Front desk24 hour
Baggage storage
Convenience store
Self-service laundry
Elevator
Housekeepingdaily
Parking & transportation
Parkingextra charge
Show details
Self parkingextra charge
EV charger
Local shuttle
Business & events
Business center
Meeting rooms
Pets
Pet-friendlyextra charge
Dogs allowed
Cats allowed
Rooms
Air conditioning
Refrigerator
Microwave
Coffee maker
Food & drink
Restaurant
Bar
Table service
Room service
Breakfastextra charge
Show details
Vending machines
Activities
Bicycle rentalextra charge
Casino
Children
Kid-friendly
Pools
Indoor pool
Hot tub
Wellness
Fitness centerfree
No spa
Accessibility
Accessible
Accessible parking
Accessible elevator
Accessible pool
Languages spoken
Arabic
English
Spanish
Bathrooms
Private bathroom
Bathtub in some rooms
Shower
Sustainability
This hotel was certified as environmentally sustainable by an independent organization recognized by Google. Learn more
Eco certifications
ISO 14001
ISO 50001
ISO 9001
Water conservation
Water-efficient toilets and showers
Towel and linen reuse program
Sustainable sourcing
Vegetarian meals
Energy efficiency
Energy conservation program
Energy use audited by an independent organization
Energy-efficient heating and cooling systems
Energy-efficient lighting
Energy-saving thermostats
Waste reduction
Recycling program
Food waste reduction program
Soap and toiletry donation program
Safely disposes of electronics, batteries, and lightbulbs
Water bottle filling stations
No Styrofoam food containers
No single-use plastic straws

"""

### User Prompt

In [4]:
user_prompt = """Consider you are the manager of Hotel Orrington. 
                On a fine day the bus named infinity bus consisting of people named "infinity times 1" to "infinity times 10". 
                Let us know if you can accommodate the people travelling in the infinity bus."""    
                
user_prompt = """Consider you are the manager of Hotel Hilton Garden Inn Chicago North Shore/Evanston. 
                On a fine day the bus named infinity bus consisting of people named "infinity times 1" to "infinity times 10". 
                Let us know if you can accommodate the people travelling in the infinity bus."""               

## Idea

In [5]:

messages = []
# query = input("Hi, how can I assist you.") # hotels with 3 rooms 2 bathroom for 3 people for 3 nights
query = user_prompt
messages.append(query)
model = AzureChatOpenAI(azure_deployment="gpt-4o", api_version="2024-10-21",  temperature=0, max_tokens=512, timeout=None, max_retries=2)


In [6]:
def info_identifier(messages, model):
    "Takes in the roles and extracts relevant information from the user query"
    template = """You are an advanced AI assistant specializing in linguistic analysis, text refinement, and **detailed information extrapolation**.  
    Your task is to process the user's query by **expanding on key details, providing additional relevant context**, and returning **only the enhanced and extrapolated version of the query** with improved clarity and structure.

    ### **Process:**
    1. **Named Entity Recognißtion (NER) & Context Expansion**:  
    - Identify and preserve proper nouns (e.g., names of people, places, organizations).  
    - Recognize temporal expressions (dates, times, durations) and expand their relevance if applicable.  
    - Retain numerical values (quantities, percentages, monetary values) and clarify their significance.  
    - Maintain specialized terms (domain-specific words requiring emphasis) while elaborating on their implications.  

    2. **Part-of-Speech (POS) Analysis & Refinement**:  
    - Ensure precise usage of nouns, pronouns, verbs, adjectives, and adverbs.  
    - Maintain sentence coherence by properly structuring prepositions, conjunctions, and articles.  

    3. **Core Information Extraction & Expansion**:  
    - Identify key ideas and extrapolate additional relevant information to provide a more detailed response.  
    - Remove redundancy while **enhancing completeness** of the original intent.  
    - Ensure logical structure and coherence while expanding descriptions and background context.  

    4. **Rephrasing, Enhancement, and Extrapolation**:  
    - Improve clarity and conciseness while **adding missing details** that would enrich understanding.  
    - Expand abbreviations, provide implicit details explicitly, and ensure a thorough description.  
    - Use a **comprehensive and well-structured narrative** that conveys the most complete possible form of the query.  

    ### **Output Requirement:**  
    - **Return only the extrapolated and enhanced query** as a well-structured, refined, and detailed text.  
    - Do not include explanations, metadata, or intermediate breakdowns—only the final, fully developed query containing all the information you have learned so far.  

    Conversation history: {messages}"""


    # print(messages)
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    chain = prompt_perspectives | model
    with get_openai_callback() as cb:
        response = chain.invoke(
            {
                "messages": str(messages[0]),
            }
        )
        print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")
    messages.append("Orchestrator output: " + response.content)
    return response.content, cb.total_cost

def generate_answer(messages, query):
    "Takes in Information, Updates user query and generates an answer"

    # print(messages)
    info = HOTEL_INFO

    template = """You are an AI assistant responsible for generating answers based on the given information.  
    You must rely strictly on the provided details and the updated user query—**do not use any external knowledge. Answer Exactly what is being asked in {user_query}**  

    ### **Input Sources:**  
    - **Provided Information:** {retrieved_info}  
    - **Updated User Query:** {updated_query}  

    ### **Task:**  
    - Analyze the retrieved information in relation to the updated query.  
    - Generate a **clear, concise, and accurate response** solely based on the given details.  
    - Ensure the response is well-structured and directly addresses the query.  

    ### **Output Requirement:**  
    - **Return only the final generated answer** without any additional commentary or explanation.  

    """

    
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    chain = prompt_perspectives | model
    with get_openai_callback() as cb:
        response = chain.invoke(
            {
                "retrieved_info": str(info),
                "updated_query": str(messages[2]),
                "user_query": messages[0]
            }
        )
        print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")
    messages.append("Answer: " + response.content)
    return response.content, cb.total_cost


In [7]:
print("USER QUERY")
print(messages[0])

# print("\n\n\n\nQUERY BASED ROLES OF LLM:")
# query_based_role_player(messages, model)

# responsibilities = messages[1]
# responsibilities = responsibilities.split("\n")
# responsibilities = responsibilities[2:-2]
# messages[1] = "".join(responsibilities)
# print(messages[1])

# Add Hotel Into
messages.append(HOTEL_INFO) # Hotel Info is stored in messgaes[2]

# print("\n\n\n\nINFORMATION BASED UPDATED ROLES OF LLM")
# info_based_role_player(messages, model)

# responsibilities = messages[]
# responsibilities = responsibilities.split("\n")
# responsibilities = responsibilities[2:-2]
# messages[1] = "".join(responsibilities)
# print(messages[1])

print("\n\n\n\nUSER QUERY POST-MORTEM")
info_identifier(messages, model)
print(messages[2])



print("\n\n\n\nFINAL OUTPUT")
generate_answer(messages, model)
print(messages[3])

USER QUERY
Consider you are the manager of Hotel Hilton Garden Inn Chicago North Shore/Evanston. 
                On a fine day the bus named infinity bus consisting of people named "infinity times 1" to "infinity times 10". 
                Let us know if you can accommodate the people travelling in the infinity bus.




USER QUERY POST-MORTEM
Total Cost (USD): $0.000000
Orchestrator output: As the manager of the Hilton Garden Inn Chicago North Shore/Evanston, could you confirm whether your hotel has the capacity to accommodate a group of 10 individuals traveling together on the "Infinity Bus"? The group consists of people identified by the names "Infinity Times 1" through "Infinity Times 10." Please provide details regarding room availability, potential group booking arrangements, and any additional accommodations or services that may be required to ensure a comfortable stay for all members of the group.




FINAL OUTPUT
Total Cost (USD): $0.000000
Answer: Based on the provided infor